In [2]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

In [3]:
entries = []
file_name = ''
with open(file_name, 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        entries.append('### Question: ' + entry['question'] + '\n' + '### Solution: ' + '\n' + entry['solution'] + '\n')

In [9]:
chunk_entries = []

batch_size = 32
for i in range(0, len(entries), batch_size):
    if i + batch_size < len(entries):
        chunk_entries.append(entries[i: i+batch_size])
    else:
        chunk_entries.append(entries[i:])

In [ ]:
path = ''

model = AutoModelForCausalLM.from_pretrained(path, device_map='auto', torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(path, padding_side='left')

In [6]:
prompt = '''You will play the role of a code implementer, writing a complete code based on the given problem and the step-by-step analysis of the problem. 
Your code must strictly follow the analysis steps provided and should not include your own opinions.
'''

In [12]:
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
output_file = open("/home/bxu/EXP/Remake-MCTS/data/nl2sl/tot-gemma-translation.json", "w", encoding="utf-8")

for chunk in tqdm(chunk_entries):
    # messages = [
    #         [
    #             {"role": "system", "content": prompt},
    #             {"role": "user", "content": entry}
    #         ]

    #         # prompt + entry
    #     for entry in chunk
    # ]
    # 使用 batch 形式处理消息
    # texts = [
    #     tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)
    #     for message in messages
    # ] # For Llama and Qwen

    texts = [prompt + '\n' + entry for entry in chunk] # For gemma which did not support chat template

    # 批量将文本编码为模型输入
    model_inputs = tokenizer(texts, return_tensors="pt", padding=True).to(model.device)

    # 生成输出（批量推理）
    generated_ids = model.generate(
        input_ids=model_inputs.input_ids,
        attention_mask=model_inputs.attention_mask,
        max_new_tokens=512,
    )
    generated_ids_trimmed = [
    output_ids[len(input_ids):]  # 从生成的 token 中移除输入的 token
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 解码输出
    generated_responses = tokenizer.batch_decode(generated_ids_trimmed, skip_special_tokens=True)

    # 打印每个输入的生成结果
    for query, response in zip(chunk, generated_responses):
        output_file.write(json.dumps({'question': query, 'solution': response}, ensure_ascii=False) + "\n")
    
    output_file.flush()

output_file.close()